In [1]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

# Load a pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
model.eval()

# ImageNet class labels
from torchvision.models import ResNet50_Weights
weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()
categories = weights.meta["categories"]


c:\Users\DELL\miniconda3\envs\face-detection\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\DELL\miniconda3\envs\face-detection\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
def predict(image_path):
    img = Image.open(image_path).convert("RGB")
    batch = preprocess(img).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        outputs = model(batch)
    probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
    top_prob, top_catid = torch.topk(probabilities, 1)
    return categories[top_catid], top_prob.item()


In [4]:
dog_images = [
    "dog1.jpg", "dog2.jpg", "dog3.jpg", "dog4.jpg", "dog5.jpg"
]

misclassified = []
for img_path in dog_images:
    pred, prob = predict(img_path)
    if "dog" not in pred.lower():   # Check if prediction is not a dog
        misclassified.append((img_path, pred, prob))

print("Misclassified Dog Images:")
for img, pred, prob in misclassified:
    print(f"{img} -> Predicted: {pred}, Confidence: {prob:.2f}")


Misclassified Dog Images:
dog1.jpg -> Predicted: Chihuahua, Confidence: 0.73
dog2.jpg -> Predicted: miniature poodle, Confidence: 0.81
dog3.jpg -> Predicted: Tibetan terrier, Confidence: 0.46
dog4.jpg -> Predicted: handkerchief, Confidence: 0.10
dog5.jpg -> Predicted: Pomeranian, Confidence: 1.00
